# Checking the Completeness of Scholars@TAMU Data  
**Filename:** cleaning.ipynb  
**Path:** TAMIDS/Code/Scholars@TAMU Data/completeness.ipynb  
**Created Date:** 18 March 2022, 16:34 

In [21]:
from IPython.display import Markdown, display, HTML
from typing import Dict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json

pd.options.display.float_format = '{:,.3f}'.format
pd.options.styler.format.precision = 2
pd.options.styler.format.thousands = ','
pd.options.display.max_columns = 50
pd.options.display.max_rows = 150
plt.style.use('seaborn-darkgrid')

# General Markdown Formatting Functions

def printmd(string, level=1):
    header_level = '#'*level + ' '
    display(Markdown(header_level + string))

## Scholars@TAMU

In [1]:
base_path = "../../Data/Scholars@TAMU"

!tree "$base_path"

../../Data/Scholars@TAMU
├── courses
│   ├── course_overview.pickle
│   └── raw_course_overview.pickle
├── grants_and_awards
│   ├── awards_overview.pickle
│   ├── grants_abstract.pickle
│   ├── grants_overview.pickle
│   ├── raw_awards_overview.pickle
│   ├── raw_grants_abstract.pickle
│   └── raw_grants_overview.pickle
├── organizations
│   ├── organizations_external.pickle
│   ├── organizations_local.pickle
│   ├── raw_organizations_external.pickle
│   └── raw_organizations_local.pickle
├── people
│   ├── people_affiliation.pickle
│   ├── people_education.pickle
│   ├── people_overview.pickle
│   ├── people_subject_areas.pickle
│   ├── raw_people_affiliation.pickle
│   ├── raw_people_education.pickle
│   ├── raw_people_overview.pickle
│   └── raw_people_subject_areas.pickle
└── publications
    ├── publications_abstract.pickle
    ├── publications_author_institutions.pickle
    ├── publications_overview.pickle
    ├── publications_subject_journal_wos.pickle
    ├── publications_unsd

In [23]:
def count_types(df: pd.DataFrame, percentage=False, plot=False, show_table=False, color_map:str=False) -> pd.DataFrame:
    new_df = pd.DataFrame()
    for column in df:
        new_df[column] = df[column].apply(lambda x: "nan" if pd.isna(x) else type(x).__name__)
    new_df = new_df.apply(pd.value_counts)

    if percentage:
        new_df = new_df / new_df.sum() * 100

    if plot:
        new_df.T.plot.bar()
        plt.yscale("log")
        plt.rcParams["figure.figsize"]
        plt.show()

    if show_table:
        if color_map:
            cm = sns.light_palette(color_map, as_cmap=True)
            color_df = new_df.style.background_gradient(cmap=cm, axis=None)
            color_df = color_df.applymap(lambda x: 'color: transparent' if pd.isnull(x) else '')
            color_df = color_df.applymap(lambda x: 'background-color: transparent' if pd.isnull(x) else '')
            display(color_df)
        else:
            display(new_df)


    return new_df


In [24]:
def slice_unnamed_columns(df: pd.DataFrame):
    return df.loc[:,~df.columns.str.contains('Unnamed:')]

def calculate_completeness(df: pd.DataFrame, show_table=False, color_map:str=False) -> pd.DataFrame:
    sliced_df = slice_unnamed_columns(df)
    missing_values = sliced_df.isna().sum()
    total_rows = sliced_df.shape[0]
    availible_values = sliced_df.count()
    data_completeness = (1 - missing_values / total_rows ) *  100

    new_df = pd.DataFrame({
        'data_completeness': data_completeness,
        'total_rows': total_rows,
        'availible_values': availible_values,
        'missing_values': missing_values
    })

    if show_table:
        if color_map:
            cm = sns.color_palette(color_map, as_cmap=True)
            color_df = new_df.style.text_gradient(cmap=cm, axis=0, gmap=new_df['data_completeness'], vmin=0, vmax=100)
            display(color_df)
        else:
            display(new_df)

    return new_df

In [25]:
def col_types_to_json(df_dict:Dict[str, pd.DataFrame], folder_name: str):
    type_dict = {}
    for file_name, df in df_dict.items():
        sliced_df = slice_unnamed_columns(df)
        counted_df = count_types(sliced_df)
        type_dict[file_name] = {column: counted_df[column].idxmax() for column in counted_df}

    with open('dicts/column_types/' + folder_name + '_col_types.json', 'w') as outfile:
        json.dump(type_dict, outfile, indent=4)

In [26]:
with open('dicts/raw_filenames.json', 'r') as outfile:
    raw_filenames = json.load(outfile)

## ./people

In [27]:
people_path = base_path + "/people"

people_raw = {picklename: pd.read_pickle(people_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['people']}

In [28]:
for name, df in people_raw.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### people_affiliation

,uid,uin,people_uri,people_api,position_dept_id
float,nan,nan,nan,nan,"5,912.00"
int,nan,"5,944.00",nan,nan,nan
nan,nan,nan,nan,nan,32.00
str,"5,944.00",nan,"5,944.00","5,944.00",nan


### people_education

,uid,uin,people_uri,people_api,external_org_id,granting_school_name,degreeuri_text,year,major
float,nan,"9,118.00",nan,nan,"9,120.00",nan,nan,nan,nan
int,nan,nan,nan,nan,nan,nan,nan,"8,521.00",nan
nan,3.00,3.00,3.00,3.00,1.00,nan,459.00,509.00,"1,398.00"
str,"9,118.00",nan,"9,118.00","9,118.00",nan,"9,121.00","8,662.00",91.00,"7,723.00"


### people_overview

,uid,uin,people_uri,people_api,status,dept_id,lastname,middle,firstname,preferred_title,email,overview,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
float,nan,nan,nan,nan,nan,"4,859.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
int,nan,"4,859.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,1.00,1.00,1.00,"1,532.00",1.00,6.00,130.00,"3,064.00","4,821.00","4,829.00","4,837.00","4,842.00","4,845.00","4,847.00","4,850.00","4,854.00","4,855.00","4,855.00","4,856.00","4,858.00","4,858.00","4,858.00","4,858.00","4,858.00","4,859.00","4,859.00"
str,"4,860.00",1.00,"4,860.00","4,860.00","4,859.00",nan,"4,859.00","3,328.00","4,859.00","4,854.00","4,730.00","1,796.00",39.00,31.00,23.00,18.00,15.00,13.00,10.00,6.00,5.00,5.00,4.00,2.00,2.00,2.00,2.00,2.00,1.00,1.00


### people_subject_areas

,uid,uin,people_uri,people_api,research_areas
int,nan,"7,328.00",nan,nan,nan
nan,nan,nan,nan,nan,"4,226.00"
str,"7,328.00",nan,"7,328.00","7,328.00","3,102.00"


In [29]:
people_stats = {}

for name, df in people_raw.items():
    printmd(name, 3)
    people_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### people_affiliation

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"5,944","5,944",0
uin,100.00,"5,944","5,944",0
people_uri,100.00,"5,944","5,944",0
people_api,100.00,"5,944","5,944",0
position_dept_id,99.46,"5,944","5,912",32


### people_education

,data_completeness,total_rows,availible_values,missing_values
uid,99.97,"9,121","9,118",3
uin,99.97,"9,121","9,118",3
people_uri,99.97,"9,121","9,118",3
people_api,99.97,"9,121","9,118",3
external_org_id,99.99,"9,121","9,120",1
granting_school_name,100.00,"9,121","9,121",0
degreeuri_text,94.97,"9,121","8,662",459
year,94.42,"9,121","8,612",509
major,84.67,"9,121","7,723","1,398"


### people_overview

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"4,860","4,860",0
uin,100.00,"4,860","4,860",0
people_uri,100.00,"4,860","4,860",0
people_api,100.00,"4,860","4,860",0
status,99.98,"4,860","4,859",1
dept_id,99.98,"4,860","4,859",1
lastname,99.98,"4,860","4,859",1
middle,68.48,"4,860","3,328","1,532"
firstname,99.98,"4,860","4,859",1
preferred_title,99.88,"4,860","4,854",6


### people_subject_areas

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"7,328","7,328",0
uin,100.00,"7,328","7,328",0
people_uri,100.00,"7,328","7,328",0
people_api,100.00,"7,328","7,328",0
research_areas,42.33,"7,328","3,102","4,226"


In [30]:
col_types_to_json(people_raw, 'people')

## ./courses

In [31]:
courses_path = base_path + "/courses"

courses_raw = {picklename: pd.read_pickle(courses_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['courses']}

In [32]:
for name, df in courses_raw.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### course_overview

,uid,uin,people_uri,people_api,course,course_name,acad_dept,year
int,nan,"70,297.00",nan,nan,nan,1.00,nan,"70,259"
nan,nan,nan,nan,nan,nan,nan,nan,33
str,"70,297.00",nan,"70,297.00","70,297.00","70,297.00","70,296.00","70,297.00",5


In [33]:
courses_stats = {}

for name, df in courses_raw.items():
    printmd(name, 3)
    courses_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### course_overview

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"70,297","70,297",0
uin,100.00,"70,297","70,297",0
people_uri,100.00,"70,297","70,297",0
people_api,100.00,"70,297","70,297",0
course,100.00,"70,297","70,297",0
course_name,100.00,"70,297","70,297",0
acad_dept,100.00,"70,297","70,297",0
year,99.95,"70,297","70,264",33


In [34]:
col_types_to_json(courses_raw, 'courses')

## ./grant_award

In [35]:
grants_path = base_path + "/grants_and_awards"

grants_raw = {picklename: pd.read_pickle(grants_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['grants_and_awards']}

In [36]:
for name, df in grants_raw.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### awards_overview

,uid,uin,people_uri,people_api,award_id,year,award_name,authoritative_name_id
float,nan,nan,nan,nan,"4,199.00","4,199.00",nan,"4,193.00"
int,1.00,"10,053.00",nan,nan,nan,nan,1.00,nan
nan,nan,nan,nan,nan,"5,855.00","5,855.00","5,861.00","5,861.00"
str,"10,053.00",1.00,"10,054.00","10,054.00",nan,nan,"4,192.00",nan


### grants_abstract

,grant_uri,grant_api,abstract,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42
int,nan,nan,nan,17,3,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,nan,nan,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,"2,970.00","5,474","5,491","5,502.00","5,512.00","5,532.00","5,548.00","5,558.00","5,579.00","5,606.00","5,611.00","5,617.00","5,633","5,650","5,673.00","5,683.00","5,700.00","5,707.00","5,719.00","5,728.00","5,739.00","5,745.00","5,751.00","5,755.00","5,763.00","5,765.00","5,766","5,768.00","5,768.00","5,772","5,773.00","5,787.00","5,789.00","5,790.00","5,794.00","5,795.00","5,798.00","5,798.00","5,798.00","5,798.00","5,798.00"
str,"5,802.00","5,802.00","2,832.00",311,308,300.00,290.00,270.00,254.00,244.00,223.00,196.00,191.00,185.00,168,151,129.00,119.00,102.00,95.00,83.00,74.00,63.00,57.00,51.00,47.00,39.00,37.00,35,34.00,34.00,29,29.00,15.00,13.00,12.00,8.00,7.00,4.00,4.00,4.00,4.00,4.00


### grants_overview

,uid,uin,people_uri,people_api,grant_uid,grant_uri,grant_api,start_year,end_year,role,grant_awarded_by_external_org_id,title,Unnamed: 12,Unnamed: 13
float,nan,nan,nan,nan,1.00,nan,nan,"5,752.00","5,729.00",nan,"5,800.00",nan,nan,nan
int,nan,"5,803.00",nan,nan,9.00,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,51.00,74.00,nan,3.00,nan,"5,800.00","5,800.00"
str,"5,803.00",nan,"5,803.00","5,803.00","5,793.00","5,803.00","5,803.00",nan,nan,"5,803.00",nan,"5,803.00",3.00,3.00


In [37]:
grants_stats = {}

for name, df in grants_raw.items():
    printmd(name, 3)
    grants_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### awards_overview

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"10,054","10,054",0
uin,100.00,"10,054","10,054",0
people_uri,100.00,"10,054","10,054",0
people_api,100.00,"10,054","10,054",0
award_id,41.76,"10,054","4,199","5,855"
year,41.76,"10,054","4,199","5,855"
award_name,41.70,"10,054","4,193","5,861"
authoritative_name_id,41.70,"10,054","4,193","5,861"


### grants_abstract

,data_completeness,total_rows,availible_values,missing_values
grant_uri,100.00,"5,802","5,802",0
grant_api,100.00,"5,802","5,802",0
abstract,48.81,"5,802","2,832","2,970"


### grants_overview

,data_completeness,total_rows,availible_values,missing_values
uid,100.00,"5,803","5,803",0
uin,100.00,"5,803","5,803",0
people_uri,100.00,"5,803","5,803",0
people_api,100.00,"5,803","5,803",0
grant_uid,100.00,"5,803","5,803",0
grant_uri,100.00,"5,803","5,803",0
grant_api,100.00,"5,803","5,803",0
start_year,99.12,"5,803","5,752",51
end_year,98.72,"5,803","5,729",74
role,100.00,"5,803","5,803",0


In [38]:
col_types_to_json(grants_raw, 'grants_and_awards')

## ./organizations

In [39]:
orgs_path = base_path + "/organizations"

orgs_raw = {picklename: pd.read_pickle(orgs_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['organizations']}

In [40]:
for name, df in orgs_raw.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### organizations_external

,external_org_id,external_org_uri,external_org_api,authoritative_name,authoritative_name_city,authoritative_name_state,authoritative_name_country,grid_id,aa_id,Unnamed: 9
int,"95,674.00",nan,nan,nan,nan,nan,nan,nan,993,nan
nan,nan,nan,nan,1.00,473.00,"60,626.00",451.00,532.00,"94,680","95,674.00"
str,nan,"95,674.00","95,674.00","95,673.00","95,201.00","35,048.00","95,223.00","95,142.00",1,nan


### organizations_local

,[dept_id],[dept_name],[department_uri],[department_api]
int,586.00,nan,nan,nan
str,nan,586.00,586.00,586.00


In [41]:
orgs_stats = {}

for name, df in orgs_raw.items():
    printmd(name, 3)
    orgs_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### organizations_external

,data_completeness,total_rows,availible_values,missing_values
external_org_id,100.00,"95,674","95,674",0
external_org_uri,100.00,"95,674","95,674",0
external_org_api,100.00,"95,674","95,674",0
authoritative_name,100.00,"95,674","95,673",1
authoritative_name_city,99.51,"95,674","95,201",473
authoritative_name_state,36.63,"95,674","35,048","60,626"
authoritative_name_country,99.53,"95,674","95,223",451
grid_id,99.44,"95,674","95,142",532
aa_id,1.04,"95,674",994,"94,680"


### organizations_local

,data_completeness,total_rows,availible_values,missing_values
[dept_id],100.00,586,586,0
[dept_name],100.00,586,586,0
[department_uri],100.00,586,586,0
[department_api],100.00,586,586,0


In [42]:
col_types_to_json(orgs_raw, 'organizations')

## ./publications

In [43]:
pubs_path = base_path + "/publications"

pubs_raw = {picklename: pd.read_pickle(pubs_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['publications']}

In [44]:
for name, df in pubs_raw.items():
    printmd(name, 3)
    new_df = count_types(df, show_table=True, color_map='green')

### publications_abstract

,publication_uri,publication_api,abstract,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
float,nan,nan,nan,6,6,10,3,8,10,12,4,5,5,5,4,2,5,3,nan,2,1,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
int,nan,nan,nan,57,44,38,27,24,17,20,18,12,12,17,13,9,10,2,3.00,5,2,1,2,2.00,2.00,3.00,5.00,nan,3.00,2.00,2.00,nan,1.00,1.00,nan,nan,nan,nan,2.00,1.00,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,"55,692.00","215,846","217,061","217,939","218,691","219,398","220,013","220,544","220,987","221,358","221,656","221,926","222,168","222,346","222,478","222,596","222,697.00","222,760","222,816","222,893","222,938","222,968.00","223,001.00","223,025.00","223,043.00","223,060.00","223,065.00","223,079.00","223,086.00","223,096.00","223,104.00","223,112.00","223,122.00","223,127.00","223,132.00","223,133.00","223,139.00","223,140.00","223,143.00","223,147.00","223,147.00","223,150.00","223,152.00","223,156.00","223,162.00","223,163.00","223,166.00","223,171.00","223,173.00","223,173.00","223,173.00","223,173.00","223,174.00","223,174.00","223,174.00","223,176.00","223,176.00","223,176.00","223,176.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00","223,177.00"
str,"223,178.00","223,178.00","167,486.00","7,269","6,067","5,191","4,457","3,748","3,138","2,602","2,169","1,803","1,505","1,230",993,821,685,577,478.00,411,359,283,237,208.00,175.00,150.00,130.00,118.00,110.00,97.00,90.00,82.00,73.00,65.00,56.00,51.00,46.00,45.00,37.00,37.00,34.00,31.00,31.00,28.00,26.00,22.00,16.00,15.00,12.00,7.00,5.00,5.00,4.00,5.00,4.00,4.00,4.00,2.00,2.00,2.00,2.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


### publications_author_institutions

,publication_uri,publication_api,organisation,city,country
int,nan,nan,nan,1.00,nan
nan,nan,nan,1.00,nan,nan
str,"950,084.00","950,084.00","950,083.00","950,083.00","950,084.00"


### publications_overview

,people_uid,uin,people_uri,people_api,dept_id,publication_uid,publication_uri,publication_api,doi,issn,eissn,journal_title,month,year,begin_page,end_page,volume,issue,publisher,publication_type,publication_title,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
datetime,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,14,17,85,"6,643",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
float,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,345,337,21,6,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
int,nan,"223,178.00",nan,nan,"223,178.00",nan,nan,nan,nan,nan,nan,1.00,"211,518.00","222,640.00","190,524","181,694","185,551","141,722",1.00,nan,7.00,3.00,3.00,1.00,1.00,1.00,1.00,3.00,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,"43,511.00","34,804.00","81,821.00","28,574.00","11,656.00",537.00,"18,454","29,315","34,059","68,192","41,309.00",2.00,15.00,"223,060.00","223,116.00","223,165.00","223,172.00","223,174.00","223,174.00","223,174.00","223,174.00","223,177.00"
str,"223,178.00",nan,"223,178.00","223,178.00",nan,"223,178.00","223,178.00","223,178.00","179,667.00","188,374.00","141,357.00","194,603.00",4.00,1.00,"13,833","11,801","3,419","6,586","181,868.00","223,176.00","223,156.00",115.00,59.00,12.00,5.00,3.00,3.00,1.00,4.00,1.00
time,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,8,14,43,29,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### publications_subject_journal_wos

,publication_uri,publication_api,wos_id,wos_research_area_id,keyword
float,nan,nan,nan,"194,946.00",nan
nan,nan,nan,"12,475.00","12,475.00","12,475.00"
str,"207,421.00","207,421.00","194,946.00",nan,"194,946.00"


### publications_unsdg

,publication_uri,publication_api,category_sdg_id,name
int,nan,nan,"14,154.00",nan
str,"14,154.00","14,154.00",nan,"14,154.00"


In [45]:
pubs_stats = {}

for name, df in pubs_raw.items():
    printmd(name, 3)
    pubs_stats[name] = calculate_completeness(df, show_table=True, color_map='light:red_r')

### publications_abstract

,data_completeness,total_rows,availible_values,missing_values
publication_uri,100.00,"223,178","223,178",0
publication_api,100.00,"223,178","223,178",0
abstract,75.05,"223,178","167,486","55,692"


### publications_author_institutions

,data_completeness,total_rows,availible_values,missing_values
publication_uri,100.00,"950,084","950,084",0
publication_api,100.00,"950,084","950,084",0
organisation,100.00,"950,084","950,083",1
city,100.00,"950,084","950,084",0
country,100.00,"950,084","950,084",0


### publications_overview

,data_completeness,total_rows,availible_values,missing_values
people_uid,100.00,"223,178","223,178",0
uin,100.00,"223,178","223,178",0
people_uri,100.00,"223,178","223,178",0
people_api,100.00,"223,178","223,178",0
dept_id,100.00,"223,178","223,178",0
publication_uid,100.00,"223,178","223,178",0
publication_uri,100.00,"223,178","223,178",0
publication_api,100.00,"223,178","223,178",0
doi,80.50,"223,178","179,667","43,511"
issn,84.41,"223,178","188,374","34,804"


### publications_subject_journal_wos

,data_completeness,total_rows,availible_values,missing_values
publication_uri,100.00,"207,421","207,421",0
publication_api,100.00,"207,421","207,421",0
wos_id,93.99,"207,421","194,946","12,475"
wos_research_area_id,93.99,"207,421","194,946","12,475"
keyword,93.99,"207,421","194,946","12,475"


### publications_unsdg

,data_completeness,total_rows,availible_values,missing_values
publication_uri,100.00,"14,154","14,154",0
publication_api,100.00,"14,154","14,154",0
category_sdg_id,100.00,"14,154","14,154",0
name,100.00,"14,154","14,154",0


In [46]:
col_types_to_json(pubs_raw, 'publications')

## Combined

In [47]:
combined_dict = {
    'people': pd.concat(people_stats.values(), axis=0, keys=people_stats.keys()),
    'courses': pd.concat(courses_stats.values(), axis=0, keys=courses_stats.keys()),
    'grants_and_awards': pd.concat(grants_stats.values(), axis=0, keys=grants_stats.keys()),
    'organizations': pd.concat(orgs_stats.values(), axis=0, keys=orgs_stats.keys()),
    'publications': pd.concat(pubs_stats.values(), axis=0, keys=pubs_stats.keys())
}

combined_df = pd.concat(combined_dict.values(), axis=0, keys=combined_dict.keys())

combined_df

data_completeness  \
people            people_affiliation               uid                                         100.000   
                                                   uin                                         100.000   
                                                   people_uri                                  100.000   
                                                   people_api                                  100.000   
                                                   position_dept_id                             99.462   
                  people_education                 uid                                          99.967   
                                                   uin                                          99.967   
                                                   people_uri                                   99.967   
                                                   people_api                                   99.967   
                                                   external_org_id                              99.989   
                                                   granting_school_name                        100.000   
                                                   degreeuri_text                               94.968   
                                                   year                                         94.419   
                                                   major                                        84.673   
                  people_overview                  uid                                         100.000   
                                                   uin                                         100.000   
                                                   people_uri                                  100.000   
                                                   people_api                                  100.000   
                                                   status                                       99.979   
                                                   dept_id                                      99.979   
                                                   lastname                                     99.979   
                                                   middle                                       68.477   
                                                   firstname                                    99.979   
                                                   preferred_title                              99.877   
                                                   email                                        97.325   
                                                   overview                                     36.955   
                  people_subject_areas             uid                                         100.000   
                                                   uin                                         100.000   
                                                   people_uri                                  100.000   
                                                   people_api                                  100.000   
                                                   research_areas                               42.331   
courses           course_overview                  uid                                         100.000   
                                                   uin                                         100.000   
                                                   people_uri                                  100.000   
                                                   people_api                                  100.000   
                                                   course                                      100.000   
                                                   course_name                                 100.000   
                                                   acad_d